In [1]:
!pip3 install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 314 kB/s 


In [2]:
import json
import implicit
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
import psycopg2
from scipy.sparse.linalg import spsolve

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [3]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

##Log 에서 사용자 조회기록 가져오기

In [4]:
db = load_db()
cursor = db.cursor()

In [5]:
load_like_log_sql = f'SELECT member_id, place_id FROM place_like'
load_click_log_sql = f'SELECT member_id, place_id FROM places_log '  

In [6]:
cursor.execute(load_click_log_sql)
result = cursor.fetchall()

###survey result 활용하기

In [10]:
survey_sql = f'select member_id, result from survey_result'
cursor.execute(survey_sql)
survey_result = cursor.fetchall()

In [21]:
survey_df = pd.DataFrame(survey_result,columns = ['member_id','survey'])

In [22]:
survey_df['len'] = survey_df['survey'].apply(lambda x: len(x))

In [24]:
survey_df = survey_df.loc[survey_df['len']>5 , ['member_id','survey']]

In [28]:
survey_df.head(3)

,member_id,survey
0,184,"[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]"
2,2202,"[51, 16, 14, 15, 9, 5, 37, 22, 50, 69]"
3,1725,"[5, 6, 7, 8, 9, 11, 12, 13, 50]"


###survey_result unique한 Row로 변환

In [32]:
entire_list = []
for item in survey_df.values:
  #print(item)
  for place in item[1]:
    entire_list.append([item[0],place])
  

In [34]:
survey_df2 = pd.DataFrame(entire_list,columns = ['member_id','place'])

In [36]:
survey_df2.sample(2)

,member_id,place
157,2847,214
48,2155,69


In [ ]:
result

###좋아요, 클릭 로그 활용하기

In [41]:
#load_click_log_sql = f'SELECT member_id, place_id FROM places_log '  
load_like_log_sql = f'SELECT member_id, place_id FROM place_like'
cursor.execute(load_like_log_sql)
like_result = cursor.fetchall()

In [42]:
like_df = pd.DataFrame(data=like_result,columns = ['member_id','place'])
like_df.head(3)

,member_id,place
0,1725,30
1,184,149
2,184,334


In [43]:
load_click_log_sql = f'SELECT member_id, place_id FROM places_log '  
cursor.execute(load_click_log_sql)
click_result = cursor.fetchall()

In [44]:
click_df = pd.DataFrame(data=click_result,columns = ['member_id','place'])
click_df.head(3)

,member_id,place
0,154,65
1,154,66
2,154,67


##세 개에서 가져온거 합치기

In [45]:
survey_df.head(1)

,member_id,survey
0,184,"[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]"


In [46]:
like_df.head(1)

,member_id,place
0,1725,30


In [47]:
click_df.head(1)

,member_id,place
0,154,65


In [51]:
tour_data = pd.concat([survey_df2,like_df,click_df],axis=0)

In [52]:
tour_data

,member_id,place
0,184,32
1,184,19
2,184,13
3,184,6
4,184,33
...,...,...
58,184,519
59,184,121
60,184,8
61,1546,61


In [53]:
tour_data.sort_values(by='place',ascending=False)

,member_id,place
41,184,992
26,184,959
12,184,959
10,184,952
49,184,949
...,...,...
59,2662,1
37,2155,1
79,184,1
39,184,1


In [56]:
tour_data.describe()

,member_id,place
count,411.000000,411.000000
mean,1726.632603,161.416058
std,1064.038352,265.771568
min,154.000000,1.000000
25%,184.000000,10.000000
50%,2188.000000,30.000000
75%,2608.000000,112.500000
max,2847.000000,992.000000


In [57]:
tour_data = tour_data.loc[tour_data['place']<300]

In [58]:
tour_data

,member_id,place
0,184,32
1,184,19
2,184,13
3,184,6
4,184,33
...,...,...
57,184,294
59,184,121
60,184,8
61,1546,61


In [107]:
tour_data.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/10.31_place_log")

###조회 횟수 및 좋아요등은 count를 반영해서 group by member,place 해야함

###추천한 아이템을 설명하기 위한 설명 데이터 불러오기

In [93]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_attraction.csv",index_col=0)
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...


##유니크한 고객ID / 유니크한 아이템 ID 를 각각 행렬의 행과 열로 만들기

In [69]:
tour_data.head(3)

,member_id,place
0,184,32
1,184,19
2,184,13


In [70]:
tour_data['quantity'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
tour_data.head(2)

,member_id,place,quantity
0,184,32,1
1,184,19,1


In [73]:
grouped_tour = tour_data.groupby(['member_id','place']).sum().reset_index()

In [74]:
grouped_tour.head(5)

,member_id,place,quantity
0,154,65,1
1,154,66,1
2,154,67,1
3,154,68,1
4,154,69,1


##Sparse Matrix 생성

In [75]:
members = list(np.sort(grouped_tour['member_id'].unique()))
places = list( grouped_tour['place'].unique())
quantity = list(grouped_tour['quantity'])

rows = grouped_tour['member_id'].astype('category').cat.codes
cols = grouped_tour['place'].astype('category').cat.codes

print(len(members))
print(len(places))

# csr: Compressed Sparse matrix by Row
purchase_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape = (len(members),len(places)))
purchase_sparse #4327 * 3650 행렬

26
62


<26x62 sparse matrix of type '<class 'numpy.int64'>'
	with 302 stored elements in Compressed Sparse Row format>

In [ ]:
quantity

###sparsity 확인

In [76]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

81.26550868486353

##train data 만들기

In [79]:
def make_train (matrix, percentage = .2):
    '''
    -----------------------------------------------------
    설명
    유저-아이템 행렬 (matrix)에서 
    1. 0 이상의 값을 가지면 1의 값을 갖도록 binary하게 테스트 데이터를 만들고
    2. 훈련 데이터는 원본 행렬에서 percentage 비율만큼 0으로 바뀜
    
    -----------------------------------------------------
    반환
    training_set: 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
    test_set:     원본 유저-아이템 행렬의 복사본
    user_inds:    훈련 데이터에서 0으로 바뀐 유저의 index
    '''
    test_set = matrix.copy()
    test_set[test_set !=0] = 1 # binary하게 만들기
    
    training_set = matrix.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
    random.seed(0)
    num_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample (nonzero_pairs, num_samples)
    
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros()
    
    return training_set, test_set, list(set(user_inds))

# 훈련, 테스트 데이터 생성
product_train, product_test, product_users_altered = make_train(purchase_sparse, 0.3)

In [80]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization = 0.1, iterations = 20)

In [81]:
alpha = 15
data = (product_train*alpha).astype('double')
model.fit(data)

  0%|          | 0/20 [00:00<?, ?it/s]

In [106]:
model.save("/content/drive/MyDrive/yeoreodigm/data_files/model")

In [82]:
item_factors, user_factors = model.item_factors, model.user_factors

In [83]:
item_factors.shape

(62, 20)

In [84]:
user_factors.shape

(26, 20)

In [85]:
predictions =  [sparse.csr_matrix(user_factors), sparse.csr_matrix(item_factors.T)]

In [86]:
predictions[0]

<26x20 sparse matrix of type '<class 'numpy.float32'>'
	with 520 stored elements in Compressed Sparse Row format>

In [87]:
predictions[1]

<20x62 sparse matrix of type '<class 'numpy.float32'>'
	with 1100 stored elements in Compressed Sparse Row format>

In [88]:
product_train

<26x62 sparse matrix of type '<class 'numpy.int64'>'
	with 211 stored elements in Compressed Sparse Row format>

##정확도 검증

In [89]:
from sklearn import metrics

In [90]:
def auc_score (test, predictions):
    '''
    fpr, tpr를 이용해서 AUC를 계산하는 함수
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr,tpr)

In [91]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    가려진 정보가 있는 유저마다 AUC 평균을 구하는 함수
    ----------------------------------------
    input
    1. training_set: make_train 함수에서 만들어진 훈련 데이터 (일정 비율로 아이템 구매량이 0으로 가려진 데이터)
    2. prediction: implicit MF에서 나온 유저/아이템 별로 나온 예측 평점 행렬
    3. altered_users: make_train 함수에서 아이템 구매량이 0으로 가려진 유저
    4. test_set: make_train함수에서 만든 테스트 데이터
    ----------------------------------------
    반환
    추천 시스템 유저의 평균 auc
    인기아이템 기반 유저 평균 auc
    '''
    # 리스트 초기화
    store_auc = []
    popularity_auc = []
    
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # 모든 유저의 아이템별 구매횟수 합
    item_vecs = predictions[1] # 아이템 latent 벡터
    
    for user in altered_users:
        training_row = training_set[user,:].toarray().reshape(-1) # 유저의 훈련데이터
        zero_inds = np.where(training_row == 0) # 가려진 아이템 Index
        
        # 가려진 아이템에 대한 예측
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # 가려진 아이템에 대한 실제값
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        
        # 가려진 아이템에 대한 popularity (구매횟수 합)
        pop = pop_items[zero_inds]
        
        # AUC 계산 
        store_auc.append(auc_score(actual, pred))
        popularity_auc.append(auc_score(actual,pop))
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  


In [92]:
calc_mean_auc(product_train, product_users_altered, predictions, product_test)
# AUC for our recommender system

(0.743, 0.885)

In [98]:
def get_items_purchased(customer_id, mf_train, customer_list, products_list, item_lookup):
    '''
    특정 유저가 구매한 목록을 보여주는 함수
    ----------------------------------------
    INPUT
    1. customer_id: 고객 ID
    2. mf_train: 훈련 데이터 평점
    3. customers_list: 훈련 데이터에 쓰인 고객 목록
    4. products_list: 훈련 데이터에 쓰인 아이템 목록
    5. item_lookup: 유니크한 아이템 ID와 설명을 담은 테이블
    '''
    cust_ind = np.where (customer_list == customer_id)[0][0]
    purchased_ind = mf_train[cust_ind,:].nonzero()[1]
    prod_codes = products_list[purchased_ind]
    
    return item_lookup.loc[item_lookup.id.isin(prod_codes)]

In [96]:
my_list = np.array([1,2,3,4,5,6,7])
np.where(my_list == 4)

(array([3]),)

In [99]:
get_items_purchased(184, product_train, np.array(members), np.array(places), attraction)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.519490,126.951090,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...
2,3,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,"바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스",33.462147,126.936424,064-783-0959,관광지,서귀포시,성산,https://api.cdn.visitjeju.net/photomng/imgpath...,1027366.0,"부모,자연경관,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, ...",2,현재 성산일출봉은 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...
4,5,천지연폭포,제주특별자치도 서귀포시 남성중로 2-15,"하늘과 땅이 만나서 이룬 연못, \r\n· 천연기념물 제163호\r\n· 천연기념물...",33.244717,126.559820,064-733-1528,관광지,서귀포시,서귀포시내,https://api.cdn.visitjeju.net/photomng/imgpath...,645261.0,"자연경관,맑음,부모,포토스팟,여름","[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, ...",10,현재 천지연폭포는 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...
6,7,새별오름,제주특별자치도 제주시 애월읍 봉성리 산 59-8,제주 서부 애월에 위치해 있으며 억새가 아름다운 오름,33.367230,126.357110,064-728-2752,관광지,제주시,애월,https://api.cdn.visitjeju.net/photomng/imgpath...,462211.0,"자연경관,봄,도보,걷기/등산,아이","[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, ...",10,"오름은 주소가 산지로 되어있어, 내비게이션 검색시 정확한 위치를 알 수 없거나, 주..."
7,8,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,"비양도, 은모래, 바다가 그려낸 그림같이 아름다운 해변",33.393845,126.239716,064-728-3981,관광지,제주시,한림,https://api.cdn.visitjeju.net/photomng/imgpath...,444882.0,"자연경관,체험,해변,맑음,액티비티,아이,여름","[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, ...",4,"제주도 서쪽에 가볼만한 해수욕장을 꼽으라 하면 단연 1,2위로 꼽아 추천하는 곳이 ..."
8,9,섭지코지,제주특별자치도 서귀포시 성산읍 섭지코지로 107,아름다운 해안이 일품인 섭지,33.430578,126.927769,064-740-6000,관광지,서귀포시,성산,https://api.cdn.visitjeju.net/photomng/imgpath...,403859.0,"자연경관,커플,맑음,포토스팟,해변,가을","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,\n 제주 동부 해안에 볼록 튀어나온 섭지코지는 성산 일출봉을 배경으로한 해안 풍경...
11,12,함덕해수욕장,제주특별자치도 제주시 조천읍 조함해안로 519-10,에매랄드처럼 빛나는 '한국의 몰디브',33.543790,126.668846,064-728-3989,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,355426.0,"자연경관,체험,해변,맑음,액티비티,아이,여름","[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, ...",4,\n 제주올레 19코스를 걷다 보면 에메랄드빛 보석을 품고 있는 함덕해수욕장을 만날...
14,15,쇠소깍,제주특별자치도 서귀포시 쇠소깍로 128,제주 올레 5코스의 끝이자 6코스의 시작인 바닷물과 민물이 만나는 비밀스런 계곡,33.252206,126.623119,064-732-1562,관광지,서귀포시,서귀포시내,https://api.cdn.visitjeju.net/photomng/imgpath...,274980.0,"자연경관,체험,커플,맑음,액티비티,휴식/힐링,여름","[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, ...",9,\n 쇠소깍은 한라산에서 흘러내려 온 물줄기가 제주도 남쪽으로 흐른다는 효돈천의 끝...
15,16,주상절리대(중문대포해안),제주특별자치도 서귀포시 이어도로 36-30,자연의 신비가 주는 웅장함에 압도되다,33.238340,126.425860,064-738-1521,관광지,서귀포시,중문,https://api.cdn.visitjeju.net/photomng/imgpath...,270792.0,"자연경관,커플,맑음,포토스팟,해변,친구","[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, ...",0,현재 주상절리대는 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...
16,17,정방폭포,제주특별자치도 서귀포시 칠십리로214번길 37,높은곳에서 바다로 떨어지는 물줄기의 시원함\r\n· 명승 제43호\r\n· 올레6코스,33.244854,126.571800,064-733-1530,관광지,서귀포시,서귀포시내,https://api.cdn.visitjeju.net/photomng/imgpath...,234780.0,"포토스팟,자연경관,커플,여름","[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",4,현재 정방폭포는 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다....


##구매하지 않은 아이템 중 추천할 아이템

In [100]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    유저의 추천 아이템 반환
    -----------------------------------------------------
    INPUT
    1. customer_id - Input the customer's id number that you want to get recommendations for
    2. mf_train: 훈련 데이터
    3. user_vecs: 행렬 분해에 쓰인 유저 벡터
    4. item_vecs: 행렬 분해에 쓰인 아이템 벡터
    5. customer_list: 평점 행렬의 행에 해당하는 고객 ID
    6. item_list: 평점 행렬의 열에 해당하는 아이템 ID
    7. item_lookup: 아이템 ID와 설명을 담은 테이블
    8. num_items: 추천할 아이템 개수
    -----------------------------------------------------
    반환    
    구매한 적이 없는 아이템 중 예측 평점이 높은 최고 n개의 추천 아이템
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0]
    pref_vec = mf_train[cust_ind,:].toarray()                   # 훈련 데이터의 실제 평점
    pref_vec = pref_vec.reshape(-1) + 1                         # 1을 더해서 환불한 것도 구매한 걸로 간주
    pref_vec[pref_vec > 1] = 0                                  # 구매한 것들을 모두 0으로 
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T)         # 추천 시스템에 기반한 예측 평점
    
    # Min-Max Scaling
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled  # 구매하지 않은 아이템에 대해서만 예측 평점이 남도록
    
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # num_items만큼 내림차순으로 평점 정렬한 index
    
    rec_list = []
    
    for index in product_idx:
        code = item_list[index] # 아이템 id
        # id와 title 담기
        rec_list.append([code, item_lookup['title'].loc[item_lookup['id'] == code].iloc[0]]) 
    
    codes = [item[0] for item in rec_list]
    title = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'id': codes, 'title': title})
    
    return final_frame[['id', 'title']]

In [131]:
np.array(members)

array([ 154,  184, 1546, 1725, 2155, 2188, 2202, 2246, 2248, 2259, 2366,
       2434, 2518, 2525, 2599, 2601, 2608, 2636, 2662, 2739, 2822, 2836,
       2844, 2845, 2846, 2847])

In [102]:
rec_items(184, product_train, user_factors, item_factors, np.array(members), np.array(places), attraction, num_items = 10)

,id,title
0,32,제주절물자연휴양림
1,204,추자도
2,210,닭머르해안길
3,51,서귀포치유의숲
4,58,성읍민속마을
5,238,서귀포천문과학문화관
6,260,송당승마장
7,11,거문오름(UNESCO 세계자연유산)
8,14,한라산국립공원
9,66,소인국테마파크


In [ ]:
np.where()

##라이브러리 제공 추천 메소드 사용하기

In [138]:
np.where(np.array(members)==184)

(array([1]),)

In [142]:
user_id = np.where(np.array(members)==2155)
model.recommend(userid=user_id,user_items=purchase_sparse[user_id],N=10,filter_already_liked_items=True)


(array([[53, 36, 38, 33, 32, 37, 21, 48, 44, 40]], dtype=int32),
 array([[0.02283749, 0.02066301, 0.01832768, 0.00852573, 0.00819446,
         0.00381242, 0.00166422, 0.        , 0.        , 0.        ]],
       dtype=float32))

##모델 저장 후 로드해서 테스트

In [ ]:
import json
import implicit
import pandas as pd
import scipy.sparse as sparse
import numpy as np

In [143]:
model.save("/content/drive/MyDrive/yeoreodigm/data_files/11.01_ALS_Model")

In [148]:
sparse.save_npz("/content/drive/MyDrive/yeoreodigm/data_files/11.01_Sparse_Matrix.npz",purchase_sparse)

In [150]:
loaded_model = model.load("/content/drive/MyDrive/yeoreodigm/data_files/11.01_ALS_Model")

In [151]:
matrix = sparse.load_npz('/content/drive/MyDrive/yeoreodigm/data_files/11.01_Sparse_Matrix.npz')

In [157]:
place_log = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/10.31_place_log.csv",index_col=0)
place_log.head(2)

array([ 154,  184, 1546, 1725, 2155, 2188, 2202, 2246, 2248, 2259, 2366,
       2434, 2518, 2525, 2599, 2601, 2608, 2636, 2662, 2739, 2822, 2836,
       2844, 2845, 2846, 2847])

In [159]:
members = np.sort(place_log['member_id'].unique())
user_id = np.where(members==2155)
loaded_model.recommend(userid=user_id,user_items=matrix[user_id],N=10,filter_already_liked_items=True)

(array([[53, 36, 38, 33, 32, 37, 21, 48, 44, 40]], dtype=int32),
 array([[0.02283749, 0.02066301, 0.01832768, 0.00852573, 0.00819446,
         0.00381242, 0.00166422, 0.        , 0.        , 0.        ]],
       dtype=float32))